<a href="https://colab.research.google.com/github/kadeng/pykurs/blob/main/notebooks/python_kurs_3_tabellen_transformieren.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Python Kurs

Dieses Notebook ist Teil 3, eines kleinen Kurses mit Übungen, in dem auf einige Themen mittleren bis einfachen Komplexitätsgrads eingegangen wird, die in generellen Einführungen zu Python manchmal zu kurz kommen, in der Praxis aber relevant sind.


## Tabellen und Tabellen Transformationen

das Thema in diesem Notebook sind Tabellen und wie man sie transformiert. Wir werden dazu das **pandas** Paket benutzen, welches man ggf. installieren muss, sowie das **re** und das **json** Paket, welche zu Python gehören.

Da dieses Notebook ohne weitere zugehörige Dateien lauffähig sein soll, verwenden wir zudem das io Package um CSV Dateien aus im Notebook eingebetteten Strings zu lesen.


In [1]:
import re
import json
import pandas as pd
import io
import numpy as np

In [2]:
# Display optionen setzen
# wir wollen die Daten komplett sehen
pd.set_option('display.max_columns', 100)
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 1000

### Pandas Grundlagen

pandas ist das beliebteste Paket um in Python mit Tabellen zu arbeiten. Alternative und durchaus erwähnenswerte Pakete sind noch **polars** welches etwas neuer und schneller ist. Vermutlich ist **polars** inzwischen besser, aber für **pandas** gibt es mehr Dokumentation und Hilfestellungen.



## pandas DataFrame und Series

Die zwei wichtigsten Datentypen die man in Pandas auch vom Namen her kennen sollte sind

### DataFrame ( Tabellen )

 Ein DataFrame ist eine komplette Tabelle, die Zeilen und Spalten hat. Da pandas allerdings die Daten **Spaltenorientiert** speichert, ist effektiv ein Dataframe etwas das einem **dict** ähnelt das einfach nur Spaltennamen (Strings) auf Objekte mappt die Spalten repräsentieren ( **pandas.Series** ).

neben diesen Spalten, verwaltet ein DataFrame auch noch einen **Index** mit dem einzelne Einträge in allen Spalten gefunden werden können. Dieser **Index** muss bei allen Spalten des Dataframes der gleiche sein, das heisst z.B. alle Spalten müssen die gleiche Länge haben. Standardmässig werden die Einträge in einem Dataframe oder auch einzelnen Spalten einfach von 0 aufsteigend durchnumeriert. Aber ein Index kann z.B. auch erlauben, jeder Zeile einen Namen zu geben.

**Man kann einem Dataframe ohne weiteres neue Spalten hinzufügen, Spalten löschen oder ersetze**n. Es ist allerdings **nicht ohne weiteres möglich, neue Zeilen hinzuzufügen oder Zeilen zu ersetzen**. Dazu schreibt man dann in der Regel besser einen neuen DataFrame. Das ist glücklicherweise recht einfach.

 ### Series ( Spalten / Columns )

 Eine pandas **Series repräsentiert in der Regel eine Spalte**. Allgemein ist es aber einfach eine Liste von Werten eines gemeinsamen Datentyps. Zusätzlich zu dieser Liste von Werten hat jede pandas Series noch einen **Index** der in der Regel von 0 aufsteigend numeriert ist.

 Es ist theoretisch möglich einzelne Werte in Spalten zu ändern, allerdings ist es auch da aus Effizienzgründen ratsamer lieber neue Spalten zu erzeugen und die alten Spalten komplett zu ersetzen.

#### Achtung: Zugriff auf Zeilen ( rows )

 Wenn man allerdings aus einem pandas DataFrame auf eine einzelne Zeile ( z.B. mit Hilfe von **DataFrame.rows**) zugreift, so wird auch diese Zeile als ein pandas.Series Objekt repräsentiert, mit einem Index der erlaubt auf einzelne Einträge anhand des Namens der Spalte zuzugreifen. Allerdings wird dieses Series Objekt dann kurzerhand erzeugt und ist eigentlich nur ein temporäres Objekt. Es **macht daher keinen Sinn zu versuchen auf diese Art Zeilen zu verändern**.



### Eine Problematische Tabelle

Wir beginnen mit einer sehr problematischen Tabelle, die aus einer Datenbank, Excel oder sonstwo kommen könnte. Sie hat zahlreiche Probleme die wir nacheinander angehen werden.

Wie man auch in diesem Beispiel gut sieht, wird der pandas DataFrame als
eine Art dict initialisiert.

In [3]:
daten_column_format =  {  "product_id" : [ 1.0, 2.0, 3.0, 4.0 ], # produkt-id sollte eigentlich ein int sein, sind aber floats
        "price" : [ "100,000.5 $", "1.130,5 EUR", "129.200€", "N/A. Bert fragt nach." ], # Wirrwarr an Formaten und Währungen, mit fehlenden Daten. Sollte eigentlich ein float-Wert in EUR sein
        "tested" : [ "Ja", "Nö", "Nein", "Vielleicht"], # Sollte eigentlich True, False oder None (unbekannt) sein
        "category" : [ "Gartenmoebel", "Gartenmoebel", "Möbel", "Moebel"], # Umlaute mal so mal so, sollte Gruppierungen erlauben
        "belastungstest_stufen" : [ "10,20,30", "10,20,30", "10,20", "10,20"], # kg mit denen die Möbel bei Tests belastet wurden
        "belastungstest_ergebnisse" : [ # Ergebnisse von Belastungstests in einem Freitextformat das man parsen muss
              "ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",
              "nicht OK:  10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",
              "Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität",
              "ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok"]}

problem_df = pd.DataFrame(daten_column_format)
problem_df

,product_id,price,tested,category,belastungstest_stufen,belastungstest_ergebnisse
0,1.0,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden."
1,2.0,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet"
2,3.0,129.200€,Nein,Möbel,"10,20","Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität"
3,4.0,N/A. Bert fragt nach.,Vielleicht,Moebel,"10,20","ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok"


### Spalten Datentyp konvertieren

in diesem Beispiel gibt es die Spalte "product_id", welche eigentlich ein Integer sein sollte. Mit Hilfe der **astype** Methode kann man sie in einen anderen Datentyp konvertieren, sofern das möglich ist (z.B. int zu float oder float zu int ist unproblematisch, aber str zu float muss nicht klappen ).

Dabei muss das Zielformat als ein **numpy** (importiert als "import numpy as np" ) Datentyp angegeben werden. Solche Datentypen sind z.B.

 * np.float32 und np.float64 für 32- und 64-bit floating point Zahlen
 * np.int32 und np.int64 für 32- und 64-bit integer
 * np.bool für boolean ( True/False)
 * np.object für beliebige Objekte, unter anderem für Strings

Zudem hat numpy auch Datentypen für Zeitstempel ( np.datetime64 etc.) die sind aber kompliziert da sie zusätzlich noch eine Einheit benötigen.

In [4]:
# die "pandas.Series.astype" Methode erlaubt es uns den Datentyp einer Spalte zu konvertieren
problem_df.product_id.astype(np.int64)

0    1
1    2
2    3
3    4
Name: product_id, dtype: int64

Das konvertieren funktioniert. Jetzt können wir die neue Spalte anstelle der alten Spalte in den Dataframe schreiben

In [5]:
problem_df['product_id'] = problem_df.product_id.astype(np.int64)
problem_df

,product_id,price,tested,category,belastungstest_stufen,belastungstest_ergebnisse
0,1,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden."
1,2,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet"
2,3,129.200€,Nein,Möbel,"10,20","Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität"
3,4,N/A. Bert fragt nach.,Vielleicht,Moebel,"10,20","ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok"


### Spalte mit Hilfe von Funktion transformieren

 da wir also in der Regel entweder Zeilen oder ganze Tabellen neu schreiben, tendiert man in pandas dazu, nicht mit Hilfe von "for" loops über die Zeilen oder Spalten zu iterieren, sondern **Funktionen** auf ganze **Zeilen** oder **Einzelne Gruppen von Zeilen** anzuwenden.

als nächstes würden wir gerne die **price** Spalte normalisieren, also in eine "price_eur" spalte überführen die entweder eine float-Zahl oder den float-Wert "nan" (not-a-number) beinhalten darf.

Da in diesem Fall keine einfache Textersetzung ausreicht ( wir müssen z.B. manchmal USD in EUR konvertieren, Komma- oder Punktformat detektieren etc.) müssen wir zunächst lernen wie man eine Spalte Zelle für Zelle durch eine beliebige Python-Funktion transformieren lassen kann. Das ist glücklicherweise nicht schwer mit Hilfe von [pandas.Series.apply](https://pandas.pydata.org/docs/reference/api/pandas.Series.apply.html) und [pandas.DataFrame.apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html)

**Series.apply** erlaubt es, eine normale Python Funktion auf jeden Wert in einer Series anzuwenden und aus dem resultat eine neue Spalte (mit gleichem Index ) zu erzeugen.

Fangen wir mal einfach an, indem wir "€" durch "EUR" ersetzen und "$" durch "USD"

In [6]:
# ersetze $ oder € in price
def normalize_currency_names(s):
  if "$" in s:
    s = s.replace("$", "USD")
  if "€" in s:
    s = s.replace("€", "EUR")
  return s


In [7]:
normalize_currency_names("Test das sind viele $ oder €")

'Test das sind viele USD oder EUR'

In [8]:
# Auf Spalte anwenden
problem_df.price.apply(normalize_currency_names)

0            100,000.5 USD
1              1.130,5 EUR
2               129.200EUR
3    N/A. Bert fragt nach.
Name: price, dtype: object

#### Übung

wie schreibe ich diese Spalte jetzt zurück in den Dataframe?

### Kommaformat normalisieren

Als nächstes Problem in der Preisspalte ist da die Sache mit Komma oder Punkt. Irgendjemand hat lustigerweise manchmal Komma und manchmal Punkt als Dezimalseparator verwendet, und dann noch um es ja nicht zu einfach zu machen, auch noch Tausender-Trennzeichen verwendet die ein einfaches suchen und ersetzen im Text komplett unmöglich machen. (Ach, das war ja ich.. naja, egal)..



In [9]:
# Versuchen wir das mal anhand von drei Beispielen
beispiele = list(problem_df.price[:-1])
# beachte, dass beim leztzen Fall *gar kein* Kommaseparator benutzt wird, sondern nur ein Tausender-Trennzeichen
beispiele

['100,000.5 $', '1.130,5 EUR', '129.200€']


### Ein Fall für zwei Regular Expressions

Wir versuchen mal das ganze in Regeln zu fassen:

 * Ein Dezimalseparator ist immer das letzte Trennzeichen, **und wird gefolgt von maximal zwei Zahlen**
 * Ein Tausender-Trennzeichen wird gefolgt von genau 3 Ziffern
 * Wir könnten je einen Regulären Ausdruck für Komma-Dezimalseparator und Punkt als Dezimalseparator definieren

### Warnung: Gefahr von Kopfschmerzen durch Regular Expressions

Regular Expressions sind bekannt dafür, selbst erfahrenen Programmierern Kopfschmerzen zu bereiten. Sie sind schlecht lesbar, man kann sich die vielen Sonderzeichen kaum merken und so weiter. Aber man kommt manchmal nicht um sie herum.

**Man kann sehr lange auf komplizierte Regular Expression starren ohne sie zu verstehen**. Diese Dinger sind leichter zu schreiben als zu lesen.

Kurzum: Selbst wirklich gute Programmierer werden bei Regulären Ausdrücken die eine gewisse Komplexität überschreiten, und die jemand anders schon mal benutzt haben muss, **zuerst Google fragen** und neuerdings auch **ChatGPT**.


### Übung

Versuch via Google oder ChatGPT dieses Problem zu lösen, und einen regulären Ausdruck zu finden der die obigen Beispiele korrekt zuordnet. Ich weiss nicht ob das klappt, aber ich würde es **immer zuerst versuchen** bei solchen Problemen die andere schon einmal hatten jedenfalls.

### Weiter im Text

Und jetzt versuchen wir es erstmal ohne Google / ChatGPT


In [10]:
# beliebig viele Ziffern, gefolgt von beliebig vielen WIederholungen von .ddd (wobei d für eine beliebige Ziffer steht), gefolt von einem *optionalen* Teil der mit einem Komma beginnt, und danach entweder eine oder zwei Ziffern folgen lässt
komma_muster = r"([0-9]+)(\.[0-9]{3,3})*(\,[0-9]{1,2})?"
re.search(komma_muster, '100,000.5 $')

<re.Match object; span=(0, 6), match='100,00'>

Mist, das hat nicht ganz hingehauen. Vielleicht müssen wir vorher alle Zeichen entfernen die keine Zahlen, Punkte oder Kommas sind, und können danach re.fullmatch verwenden. Oder wir sagen dem Regex, dass **vor und nach** dem Match keine Zahl, Komma oder Punkt kommen darf.

Das machen wir mal. Vor der Zahl darf entweder der Textanfang ("^") oder ein Zeichen das weder Zahl noch Punkt oder Komma ist stehen "[^0-9,.]". Die Alternative druecken wir mit | aus. Am Ende das gleiche, nur statt Textanfang waere dort das Textende ("$" in Ordnung)


### Regex Lookahead / Lookbehind



In [12]:
# beachte das ([^0-9,.]|$) am Ende, es bedeutet "entweder keine Zahl, Komma oder Punkt, oder das Ende des Texts"
# und das ([^0-9,.]|^)([0-9]+) am Anfang bedeutet "entweder keine Zahl, Komma oder Punkt, oder der Anfang des Texts"
komma_muster = r"([^0-9,.]|^)(([0-9]+)(\.[0-9]{3,3})*(\,[0-9]{1,2})?)([^0-9,.]|$)"


### Regular Expression mit Hilfe von ChatGPT verstehen

Als eine weitere Übung, nehmen wir mal die obige regular expression, und lassen sie uns von ChatGPT erklären.

Kopiere folgende Abfrage und stell sie ChatGPT:

```
Erläutere mir den folgenden regulären Ausdruck: "([^0-9,.]|^)(([0-9]+)(\.[0-9]{3,3})*(\,[0-9]{1,2})?)([^0-9,.]|$)".
Zu welchem Zweck dient er wahrscheinlich?
Auf welche Strings wird er matchen ?
Erläutere die Teilausdrücke die er beinhaltet.
```

Anschliessend testen wir den Ausdruck. Sollte ChatGPT Beispiele haben bei denen er angeblich nicht funktioniert, könnten wir die hier auch testen.

In [13]:
if not re.search(komma_muster, '100,000.5 $'):
  print("Kein Match, so soll es sein!")

Kein Match, so soll es sein!


In [14]:
if re.search(komma_muster, '1.130,5 EUR'):
  print("Ein Match, so soll es sein!")

Ein Match, so soll es sein!


In [15]:
if re.search(komma_muster, '1.130 EUR'):
  print("Ein Match, so soll es sein!")

Ein Match, so soll es sein!


### Preis Parser

Jetzt können wir endlich eine Funktion schreiben, die mit diesen Preis-Texten umgehen kann. Wie man sieht, kann sowas beliebig komplex werden. Das Muster fuer Texte mit Komma als Tausender-Trennzeichen und Punkt als Dezimalseparator funktioniert genau wie das andere, nur eben mit Punkt und Komma vertauscht.

In [16]:
def parse_preis(preis_text, eur_usd_wechselkurs=0.93):
  # Muster das auf Zahlen im Format mit Komma als Dezimalseparator passt
  komma_muster = r"([^0-9,.]|^)(([0-9]+)(\.[0-9]{3,3})*(\,[0-9]{1,2})?)([^0-9,.]|$)"
  # Muster das auf Zahlen im Format mit Punkt als Dezimalseparator passt
  punkt_muster = r"([^0-9,.]|^)(([0-9]+)(\,[0-9]{3,3})*(\.[0-9]{1,2})?)([^0-9,.]|$)"
  preis = float('nan') # Standardpreis der zurueckgegeben wird wenn sonst nichts passt. nan=not a number
  waehrung = ""
  # gibt es einen Preis mit Komma separator?
  m = re.search(komma_muster, preis_text)
  if m:
    zahl_text = m.group(2) # Wir finden die Zahl indem wir Gruppe 2 extrahieren. Pruef das gerne nach mit print statements
    # Jetzt verwandeln wir diesen Teilstring in eine float. Dazu müssen wir daraus alle Punkte
    # rausschmeissen und das Komma durch Punkt ersetzen bevor wir es in eine float wandeln
    preis = float(zahl_text.replace(".", "").replace(",", "."))
    # wir nehmen an, dass standardmässig Preise in diesem Format in EUR sind
    waehrung = "€"
  else:
    m = re.search(punkt_muster, preis_text)
    if m:
      zahl_text = m.group(2) # Wir finden die Zahl indem wir Gruppe 2 extrahieren. Pruef das gerne nach mit print statements
      # Jetzt verwandeln wir diesen Teilstring in eine float. Dazu müssen wir daraus alle Kommas
      # rausschmeissen.
      preis = float(zahl_text.replace(",", ""))
      waehrung = "$" # wir nehmen an, dass standardmässig Preise in diesem Format in Dollar sind#

  # Falls explizit die Währung genaannt wird, nehmen wir die
  if "$" in preis_text or "USD" in preis_text:
    waehrung = "$"
  elif "€" in preis_text or "EUR" in preis_text:
    waehrung = "€"
  if waehrung == "$":
    preis = preis * eur_usd_wechselkurs # wechselkurs umrechnen
  # resultat auf 2 Stellen runden
  return round(preis,2)




In [17]:
problem_df.price

0              100,000.5 $
1              1.130,5 EUR
2                 129.200€
3    N/A. Bert fragt nach.
Name: price, dtype: object

In [18]:
# probieren wir die Funktion mal aus
problem_df.price.apply(parse_preis)

0     93000.47
1      1130.50
2    129200.00
3          NaN
Name: price, dtype: float64

das sieht gut aus. Die 93000.47 kommt durch den Wechselkurs zustande. Jetzt uebernehmen wir das ganze in den Datensatz als neue Spalte. "preis_eur"

In [19]:
problem_df['preis_eur'] = problem_df.price.apply(parse_preis)
# del problem_df['price'] # alte Spalte loeschen ( del = delete )
problem_df

,product_id,price,tested,category,belastungstest_stufen,belastungstest_ergebnisse,preis_eur
0,1,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",93000.47
1,2,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",1130.50
2,3,129.200€,Nein,Möbel,"10,20","Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität",129200.00
3,4,N/A. Bert fragt nach.,Vielleicht,Moebel,"10,20","ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok",NaN


## Tabellen aufblaehen

sagen wir mal, wir wollen die Daten fuer die Belastungstests genauer auswerten. Dazu muessen wir an die einzelnen Eintraege aus belastungstest_stufen und belastungstest_ergebnisse ran.

Dazu konvertieren wir erstmal die Belastungstest Stufen jeweils in Listen.

In [20]:
def split_to_list(s):
  return [ part.strip() for part in s.split(",") ]

problem_df.belastungstest_stufen.apply(split_to_list)

0    [10, 20, 30]
1    [10, 20, 30]
2        [10, 20]
3        [10, 20]
Name: belastungstest_stufen, dtype: object

das sieht ok aus. Schreiben wir es in den Dataframe

In [21]:
problem_df['belastungstest_stufe'] = problem_df.belastungstest_stufen.apply(split_to_list)

In [22]:
problem_df

,product_id,price,tested,category,belastungstest_stufen,belastungstest_ergebnisse,preis_eur,belastungstest_stufe
0,1,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",93000.47,"[10, 20, 30]"
1,2,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",1130.50,"[10, 20, 30]"
2,3,129.200€,Nein,Möbel,"10,20","Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität",129200.00,"[10, 20]"
3,4,N/A. Bert fragt nach.,Vielleicht,Moebel,"10,20","ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok",NaN,"[10, 20]"


### Zeilen aufblaehen mit explode

Jetzt wo es eine Liste ist, koennen wir [DataFrame.explode](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html) verwenden, und Zeilen replizieren. Jede neue Zeile bekommt dabei einen Wert



In [24]:
# vorher
problem_df

,product_id,price,tested,category,belastungstest_stufen,belastungstest_ergebnisse,preis_eur,belastungstest_stufe
0,1,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",93000.47,"[10, 20, 30]"
1,2,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",1130.50,"[10, 20, 30]"
2,3,129.200€,Nein,Möbel,"10,20","Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität",129200.00,"[10, 20]"
3,4,N/A. Bert fragt nach.,Vielleicht,Moebel,"10,20","ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok",NaN,"[10, 20]"


In [25]:
#nachher
problem_df.explode("belastungstest_stufe")

,product_id,price,tested,category,belastungstest_stufen,belastungstest_ergebnisse,preis_eur,belastungstest_stufe
0,1,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",93000.47,10
0,1,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",93000.47,20
0,1,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",93000.47,30
1,2,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",1130.50,10
1,2,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",1130.50,20
1,2,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",1130.50,30
2,3,129.200€,Nein,Möbel,"10,20","Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität",129200.00,10
2,3,129.200€,Nein,Möbel,"10,20","Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität",129200.00,20
3,4,N/A. Bert fragt nach.,Vielleicht,Moebel,"10,20","ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok",NaN,10
3,4,N/A. Bert fragt nach.,Vielleicht,Moebel,"10,20","ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok",NaN,20


**Anmerkung**: Um diese Operation gewissermassen spaeter rueckgaengig zu machen kann man **groupby**, **pivot** oder **pivot_table** verwenden.

### Weitere hilfreiche DataFrame Operationen um Tabellen aufzublaehen oder (umgekehrt) gruppiert zusammenzufassen

In diesem Zusammenhang kann man sich auch gleich mit der Doku von ein paar anderen DataFrame Methoden und Workflows bekannt machen. Dabei ist es vor allem wichtig zu wissen **dass es die folgenden Funktionalitaeten ueberhaupt gibt**. Die Details schlaegt man in dem Moment nach in dem man sie benutzt.

 * [DataFrame.melt](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.melt.html) um Tabellen aufzublaehen indem Spalten in Name: Wert Paare verwandelt werden.
 * [**GroupBy: split-apply-combine Workflow**](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) ein kompletter Workflow, um Tabellen zu gruppieren und zu transformieren.
 * [DataFrame.pivot](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html) Um eine einfache Zusammenfassungstabelle zu erstellen.Weniger maechtig als Groupby-split-apply-combine, aber einfacher.
 * [DataFrame.pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html) Um Daten gruppiert zu analysieren und eine entsprechend zusammengefasste Tabelle schnell zu erstellen, wobei sich die Funktionalitaet an Excel Pivottabellen orientiert. Weniger maechtig als Groupby-split-apply-combine, aber einfacher.


### Parser fuer Belastungstest-Ergebnisse

die belastungstest_ergebnisse Spalte ist ohne Frage am kompliziertesten. Aber wir koennen bestimmt trotzdem einen oder mehrere Parser fuer sie schreiben. Fange wir mal an:



In [31]:
# Gesamtergebnis am Anfang auslesen
def gesamtergebnis(ergebnis):
  if ergebnis.lower().startswith("ok:"):
    return "OK"
  elif ergebnis.lower().startswith("nicht ok:"):
    return "FAILURE"
  else:
    return ergebnis[:ergebnis.find(":")]


In [32]:
problem_df.belastungstest_ergebnisse.apply(gesamtergebnis)

0         OK
1    FAILURE
2         OK
3         OK
Name: belastungstest_ergebnisse, dtype: object

In [74]:
problem_df['gesamtergebnis'] = problem_df.belastungstest_ergebnisse.apply(gesamtergebnis)
problem_df

,product_id,price,tested,category,belastungstest_stufen,belastungstest_ergebnisse,preis_eur,belastungstest_stufe,gesamtergebnis
0,1,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",93000.47,"[10, 20, 30]",OK
1,2,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",1130.50,"[10, 20, 30]",FAILURE
2,3,129.200€,Nein,Möbel,"10,20","Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität",129200.00,"[10, 20]",OK
3,4,N/A. Bert fragt nach.,Vielleicht,Moebel,"10,20","ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok",NaN,"[10, 20]",OK


## Zeilenweise Transformation einer Tabelle

man kann auch in Pandas eine Tabelle Zeilenweise transformieren. Das ist vor allem dann sinnvoll wenn man eine oder mehrere neue Spalten aufbauen oder existierende veraendern moechte, dabei aber Daten aus mehreren Spalten (aber nur einer Zeile ) benoetigt.

Das geht z.B. so:

In [49]:
category_id_map = {
    "Gartenmoebel" : 1,
    "Möbel" : 2,
    "Moebel" : 2
}
def transform_row(row):
  row["category_id"] = category_id_map[row["category"]]
  row["id_and_category"] = str(row["product_id"]) + "_" + str(row["category_id"])
  del row["price"]
  del row["tested"]
  return row



In [50]:
problem_df.apply(transform_row, axis=1)

,product_id,category,belastungstest_stufen,belastungstest_ergebnisse,preis_eur,belastungstest_stufe,gesamtergebnis,category_id,id_and_category
0,1,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",93000.47,"[10, 20, 30]",OK,1,1_1
1,2,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",1130.50,"[10, 20, 30]",FAILURE,1,2_1
2,3,Möbel,"10,20","Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität",129200.00,"[10, 20]",OK,2,3_2
3,4,Moebel,"10,20","ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok",NaN,"[10, 20]",OK,2,4_2


Das erlaubt uns jetzt, eine Transformation der *explodierten* Tabelle.

In [61]:
def ergebnis_transformation(row):
  row['ergebnis_bei_stufe'] = None
  stufe = str(row['belastungstest_stufe'])
  belastungstest_ergebnisse = row['belastungstest_ergebnisse']
  pattern = re.escape(stufe) + r" *(kg)? *\: *(.*?) *([.,]|$)" # Regular Expression zusammenbauen. *? ist ein sogenannter non-greedy repetition operator. Er matcht so *wenige* Zeichen wie er kann. Ohne das Fragezeichen versucht er so viele zu matchen wie geht.
  m = re.search(pattern, belastungstest_ergebnisse)
  if m:
    row['ergebnis_bei_stufe'] = m.group(2)
  else:
    print(f"Pattern: {pattern} passte nicht auf {belastungstest_ergebnisse}")
  return row

In [66]:
df = problem_df.explode("belastungstest_stufe").apply(ergebnis_transformation, axis=1)
df

,product_id,price,tested,category,belastungstest_stufen,belastungstest_ergebnisse,preis_eur,belastungstest_stufe,gesamtergebnis,ergebnis_bei_stufe
0,1,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",93000.47,10,OK,100 Stunden
0,1,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",93000.47,20,OK,50 Stunden
0,1,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",93000.47,30,OK,25 Stunden
1,2,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",1130.50,10,FAILURE,10 Stunden
1,2,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",1130.50,20,FAILURE,3 Stunden
1,2,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",1130.50,30,FAILURE,Zusammenbruch
2,3,129.200€,Nein,Möbel,"10,20","Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität",129200.00,10,OK,100 Stunden
2,3,129.200€,Nein,Möbel,"10,20","Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität",129200.00,20,OK,100 Stunden
3,4,N/A. Bert fragt nach.,Vielleicht,Moebel,"10,20","ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok",NaN,10,OK,100 Stunden
3,4,N/A. Bert fragt nach.,Vielleicht,Moebel,"10,20","ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok",NaN,20,OK,10 Stunden


In [69]:
def stunden_extrahieren(ergebnis_bei_stufe):
  m = re.fullmatch("([0-9]+) ?Stunden", ergebnis_bei_stufe, re.IGNORECASE)
  if m:
    return int(m.group(1))
  else:
    return 0


In [70]:
df['stunden_ausgehalten'] = df.ergebnis_bei_stufe.apply(stunden_extrahieren)
df

,product_id,price,tested,category,belastungstest_stufen,belastungstest_ergebnisse,preis_eur,belastungstest_stufe,gesamtergebnis,ergebnis_bei_stufe,stunden_ausgehalten
0,1,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",93000.47,10,OK,100 Stunden,100
0,1,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",93000.47,20,OK,50 Stunden,50
0,1,"100,000.5 $",Ja,Gartenmoebel,"10,20,30","ok: 10 kg: 100 Stunden, 20 kg: 50 Stunden, 30 kg: 25 Stunden.",93000.47,30,OK,25 Stunden,25
1,2,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",1130.50,10,FAILURE,10 Stunden,10
1,2,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",1130.50,20,FAILURE,3 Stunden,3
1,2,"1.130,5 EUR",Nö,Gartenmoebel,"10,20,30","nicht OK: 10: 10 Stunden, 20: 3 Stunden, 30: Zusammenbruch. Kommentar: Wasserresistenz nicht getestet",1130.50,30,FAILURE,Zusammenbruch,0
2,3,129.200€,Nein,Möbel,"10,20","Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität",129200.00,10,OK,100 Stunden,100
2,3,129.200€,Nein,Möbel,"10,20","Ok: 10: 100 Stunden, 20 : 100 Stunden. Kommentar: Super Qualität",129200.00,20,OK,100 Stunden,100
3,4,N/A. Bert fragt nach.,Vielleicht,Moebel,"10,20","ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok",NaN,10,OK,100 Stunden,100
3,4,N/A. Bert fragt nach.,Vielleicht,Moebel,"10,20","ok: 10: 100 Stunden, 20:10 Stunden, 30: N/A. Kommentar: Qualität ok",NaN,20,OK,10 Stunden,10


### Resultate gruppiert zusammenfassen ( group-by-combine)

Jetzt haben wir eigentlich alle Ergebnisse beisammen. Man koennte das ganze jetzt sehr gut wieder in eine Datenbank reexportieren und dort mit Hilfe von GROUP BY statements zusammenfassen, filtern, analysieren etc.

Aber man kann das natuerlich auch mit Python und Pandas. Es ist aber etwas umstaendlich.

Zunaechst einmal: Sofern wir nur Spaltenweise zusammenfassen wollen, kann man mit Hilfe der Methoden [pandas.DataFrameGroupBy.transform](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.transform.html) oder mit [pandas.DataFrameGroupBy.agg](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.agg.html) sehr weit kommen.

Aber in unserem Fall wollen wir maximale Kontrolle. Die bekommen wir, indem wir einfach eine Liste von dictionaries aufbauen, wobei jedes dictionary eine Zeile in einer neuen Tabelle darstellt. Und diese Liste bauen wir auf, indem wir und nacheinander jede Gruppe eines **DataFrame.groupby** resultats ansehen.




In [93]:

# Diese Funktion wenden wir auf jede Gruppe an. Wir machen effektiv eine Zeile aus jeder Gruppe.
# wir koennten aber auch mehr (oder weniger) Zeilen daraus machen wenn wir wollten.
def gruppe_transform(group_rows):
  res = dict(group_rows.iloc[0]) # wir machen ein Dictionary aus der ersten Zeile
  # loeschen alle Felder die wir raushaben wollen
  del res['price']
  del res["ergebnis_bei_stufe"]
  del res['belastungstest_ergebnisse']
  # und berechnen ein paar neue, dabei koennen wir ohne weiteres Mittelwerte bilden
  # und andere Aggregatfunktionen ausfuehren, wie es uns beliebt.
  res['stunden_ausgehalten'] = json.dumps(list(group_rows.stunden_ausgehalten))
  res["stunden_ausgehalten_median"] = group_rows.stunden_ausgehalten.median()
  # hier geben wir nur eine Zeile zurueck, aber wir koennten auch (z.B. mit Hilfe von yield)
  # beliebig viele zurueckgeben
  return res


In [95]:
neue_zeilen = []
for gruppe, gruppen_df in df.groupby(["product_id", "price", "category", "preis_eur", "gesamtergebnis"]):
  neue_zeile = gruppe_transform(gruppen_df)
  neue_zeilen.append(neue_zeile)
neue_zeilen

[{'product_id': 1,
  'tested': 'Ja',
  'category': 'Gartenmoebel',
  'belastungstest_stufen': '10,20,30',
  'preis_eur': 93000.47,
  'belastungstest_stufe': '10',
  'gesamtergebnis': 'OK',
  'stunden_ausgehalten': '[100, 50, 25]',
  'stunden_ausgehalten_median': 50.0},
 {'product_id': 2,
  'tested': 'Nö',
  'category': 'Gartenmoebel',
  'belastungstest_stufen': '10,20,30',
  'preis_eur': 1130.5,
  'belastungstest_stufe': '10',
  'gesamtergebnis': 'FAILURE',
  'stunden_ausgehalten': '[10, 3, 0]',
  'stunden_ausgehalten_median': 3.0},
 {'product_id': 3,
  'tested': 'Nein',
  'category': 'Möbel',
  'belastungstest_stufen': '10,20',
  'preis_eur': 129200.0,
  'belastungstest_stufe': '10',
  'gesamtergebnis': 'OK',
  'stunden_ausgehalten': '[100, 100]',
  'stunden_ausgehalten_median': 100.0}]

In [96]:
# Anschliessend machen wir da wieder einen DataFrame draus. Fertig
pd.DataFrame.from_records(neue_zeilen)

,product_id,tested,category,belastungstest_stufen,preis_eur,belastungstest_stufe,gesamtergebnis,stunden_ausgehalten,stunden_ausgehalten_median
0,1,Ja,Gartenmoebel,"10,20,30",93000.47,10,OK,"[100, 50, 25]",50.0
1,2,Nö,Gartenmoebel,"10,20,30",1130.50,10,FAILURE,"[10, 3, 0]",3.0
2,3,Nein,Möbel,"10,20",129200.00,10,OK,"[100, 100]",100.0
